In [2]:
import psycopg2
import random
from datetime import datetime, timedelta
!pip install Faker
from faker import Faker

fake = Faker('ru_RU')

def generate_license_plate():
    letters = 'АВЕКМНОРСТУХ'
    return (
        random.choice(letters) +
        str(random.randint(100, 999)) +
        random.choice(letters) +
        random.choice(letters)
    )

try:
    # Подключение к базе данных
    conn = psycopg2.connect(
        user='bk_465842_2025',
        password='bk_465842',
        host='postgrepro.dc-edu.ru',
        port='5432',
        database='dbstud'
    )
    cursor = conn.cursor()

    cursor.execute("DROP TABLE IF EXISTS order_details")
    cursor.execute("DROP TABLE IF EXISTS taxi_order")
    cursor.execute("DROP TABLE IF EXISTS driver")
    cursor.execute("DROP TABLE IF EXISTS client")
    conn.commit()

    # Создание таблиц
    cursor.execute("""
    CREATE TABLE client (
        client_id SERIAL PRIMARY KEY,
        client_name VARCHAR(50) NOT NULL,
        client_phone VARCHAR(50) UNIQUE NOT NULL
    )""")

    cursor.execute("""
    CREATE TABLE driver (
        driver_id SERIAL PRIMARY KEY,
        car_model VARCHAR(50) NOT NULL,
        car_color VARCHAR(50),
        license_plate VARCHAR(50) UNIQUE NOT NULL
    )""")

    cursor.execute("""
    CREATE TABLE taxi_order (
        order_id SERIAL PRIMARY KEY,
        driver_id INTEGER NOT NULL REFERENCES driver(driver_id),
        client_id INTEGER NOT NULL REFERENCES client(client_id),
        order_datetime VARCHAR(50),
        pickup_address TEXT NOT NULL,
        destination_address TEXT NOT NULL
    )""")

    cursor.execute("""
    CREATE TABLE order_details (
        order_id INTEGER PRIMARY KEY REFERENCES taxi_order(order_id),
        arrival_time VARCHAR(50),
        cost INTEGER,
        payment_type VARCHAR(50) CHECK (payment_type IN ('наличные', 'карта')),
        status VARCHAR(50)
    )""")

    conn.commit()

    count = 125
    for _ in range(count):
        cursor.execute(
            "INSERT INTO client (client_name, client_phone) VALUES (%s, %s)",
            (fake.name(), fake.phone_number())
        )
    conn.commit()

    car_models = ['Toyota Camry', 'Hyundai Solaris', 'Kia Rio', 'Lada Granta',
                 'Skoda Octavia', 'Volkswagen Polo', 'Renault Logan']
    colors = ['белый', 'черный', 'серый', 'синий', 'красный']

    drivers = []
    for _ in range(count):
        cursor.execute(
            """INSERT INTO driver (car_model, car_color, license_plate)
            VALUES (%s, %s, %s) RETURNING driver_id""",
            (random.choice(car_models),
             random.choice(colors),
             generate_license_plate())
        )
        drivers.append(cursor.fetchone()[0])
    conn.commit()

    statuses = ['принят', 'завершен', 'отменен']
    payment_types = ['наличные', 'карта']

    for _ in range(count):

        order_datetime = fake.date_time_between(start_date='-90d', end_date='now')
        formatted_datetime = order_datetime.strftime('%d:%m:%y %H:%M:%S')

        cursor.execute(
            """INSERT INTO taxi_order (
                driver_id, client_id, order_datetime,
                pickup_address, destination_address
            ) VALUES (%s, %s, %s, %s, %s) RETURNING order_id""",
            (random.choice(drivers),
             random.randint(1, count),
             formatted_datetime,
             fake.street_address(),
             fake.street_address())
        )
        order_id = cursor.fetchone()[0]

        arrival_time = order_datetime + timedelta(minutes=random.randint(5, 30))
        formatted_arrival = arrival_time.strftime('%d:%m:%y %H:%M:%S')

        cursor.execute(
            """INSERT INTO order_details (
                order_id, arrival_time, cost,
                payment_type, status
            ) VALUES (%s, %s, %s, %s, %s)""",
            (order_id,
             formatted_arrival,
             random.randint(1, 2000),
             random.choice(payment_types),
             random.choices(statuses, weights=[0.2, 0.7, 0.1])[0])
        )
    conn.commit()

    print("База данных успешно заполнена")

except Exception as e:
    print(f"Ошибка: {e}")
    if 'conn' in locals():
        conn.rollback()
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()

База данных успешно заполнена:
